# Modeling – Heart Disease Prediction

Ce notebook vise à entraîner et comparer plusieurs modèles de classification
afin de prédire la présence d’une maladie cardiaque à partir des données cliniques.
Les modèles sont évalués à l’aide de métriques adaptées.


In [11]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


## Chargement du pipeline de preprocessing

Le pipeline de preprocessing sauvegardé lors de la phase précédente est chargé
afin d’assurer la cohérence entre les transformations appliquées aux données
d’entraînement et de test.

Cette approche permet de séparer clairement les étapes de preprocessing et de
modélisation, tout en garantissant la reproductibilité du projet.


In [12]:
import joblib

preprocessor = joblib.load("models/preprocessor.joblib")


## Chargement des données et séparation train/test

Les données sont rechargées et séparées en jeux d’entraînement et de test
en utilisant les mêmes paramètres que lors de la phase de preprocessing,
afin d’assurer la cohérence des résultats.


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Rechargement du dataset
df = pd.read_csv("dataset.csv")

# Suppression de l'identifiant
df = df.drop(columns=["id"])

# Séparation X / y
X = df.drop(columns=["num"])
y = df["num"]

# Split train / test (mêmes paramètres)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


## Données utilisées

Les données ont été préparées dans le notebook de preprocessing.
Les jeux d’entraînement et de test ainsi que le pipeline de preprocessing
sont utilisés afin d’entraîner les modèles sans fuite de données.


## Modèle 1 – Régression logistique (baseline)

La régression logistique est utilisée comme modèle de référence en raison de sa
simplicité et de son interprétabilité. Elle permet d’évaluer rapidement si les
variables contiennent un signal prédictif pertinent.


In [14]:
log_reg = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)

acc_lr = accuracy_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr, average="weighted")

acc_lr, f1_lr


(0.5869565217391305, 0.5614639051891325)

## Modèle 2 – Random Forest

Les Random Forest sont des modèles ensemblistes capables de modéliser des relations
non linéaires entre les variables. Ils sont généralement robustes et performants
sur des données tabulaires.



In [15]:
rf = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=200,
        random_state=42
    ))
])

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average="weighted")

acc_rf, f1_rf


(0.5869565217391305, 0.5687354137080654)

## Modèle 3 – Gradient Boosting

Le Gradient Boosting combine plusieurs modèles faibles afin d’améliorer
progressivement les performances. Il est souvent très performant pour les
problèmes de classification structurés.


In [16]:
from sklearn.ensemble import GradientBoostingClassifier

gb = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", GradientBoostingClassifier(random_state=42))
])

gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_test)

acc_gb = accuracy_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb, average="weighted")

acc_gb, f1_gb


(0.6141304347826086, 0.59259280658674)

In [17]:
results = pd.DataFrame({
    "Model": [
        "Logistic Regression",
        "Random Forest",
        "Gradient Boosting"
    ],
    "Accuracy": [
        acc_lr,
        acc_rf,
        acc_gb
    ],
    "F1-score": [
        f1_lr,
        f1_rf,
        f1_gb
    ]
})

results


,Model,Accuracy,F1-score
0,Logistic Regression,0.586957,0.561464
1,Random Forest,0.586957,0.568735
2,Gradient Boosting,0.614130,0.592593


## Comparaison des modèles

La régression logistique sert de modèle de référence mais présente des performances
limitées. Les modèles ensemblistes obtiennent de meilleurs résultats, indiquant
la présence de relations non linéaires dans les données.

Le modèle de Gradient Boosting sera retenu pour la phase d’optimisation des
hyperparamètres.
